In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp loggers.core

In [4]:
# export
# Python native modules
import os,typing
# Third party libs
from fastcore.all import *
from torch.multiprocessing import Queue
import torchdata.datapipes as dp
from fastprogress.fastprogress import *
# Local modules
from fastrl.pipes.core import *

# Loggers Core
> Core Utilties for logging in fastrl using Callbacks and DataPipes

In [5]:
# export
class LoggerBase(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None):
        self.source_datapipe = source_datapipe
        self.main_queue = Queue()
        
    def connect_source_datapipe(self,pipe):
        self.source_datapipe = pipe
        return self
        

The `LoggerBase` class outlines simply the `main_queue`. It works in combo with `LogCollector` datapipe which will add to the `main_queue` even across processes. 

In [6]:
# export
class LogCollector(dp.iter.IterDataPipe):
    def __init__(self,
         source_datapipe, # The parent datapipe, likely the one to collect metrics from
         logger_bases:List[LoggerBase] # `LoggerBase`s that we want to send metrics to
        ):
        self.source_datapipe = source_datapipe
        self.main_queues = [o.main_queue for o in logger_bases]
        
    def __iter__(self): raise NotImplementedError

Notes:

User can init multiple different logger bases if they want

We then can manually add Collectors, custom for certain pipes such as for collecting rewards. This can be put in 
full on pipelines even across workers.

The manual process might not be desireable though, so I'm thinking of revising the `add_cbs_to_pipes` where
the pipe itself literally says if it should go before, after, exclude from.

This seems... easier than defining a callback, and keeps a level of flatness to the api.





In [7]:
# export
def is_pipe_instance(pipe,cls): return isinstance(pipe,cls) 
def find_pipe_instance(main_pipe,pipe_cls):
    return find_pipes(main_pipe,partial(is_pipe_instance,cls=pipe_cls))[0]

In [8]:
# export
class Record(typing.NamedTuple):
    name:str
    value:typing.Any

In [9]:
# export
class ProgressBarLogger(LoggerBase):
    def __init__(self,
                 # This does not need to be immediately set since we need the `LogCollectors` to 
                 # first be able to reference its queues.
                 source_datapipe=None, 
                 # For this, and many LoggerBase objects, they likely will show their metrics
                 # at the end of each epoch.
                 epochs=None,
                 # For this, and many LoggerBase objects, they likely will show progress of an epoch
                 # at the end of each batch.
                 batches=None,
                 # For automatic pipe attaching, we can designate which pipe this should be
                 # referneced for information on which epoch we are on
                 epoch_on_pipe:dp.iter.IterDataPipe=None,
                 # For automatic pipe attaching, we can designate which pipe this should be
                 # referneced for information on which batch we are on
                 batch_on_pipe:dp.iter.IterDataPipe=None
                ):
        self.source_datapipe = source_datapipe
        self.main_queue = Queue()
        self.epochs = epochs
        self.batches = batches
        self.epoch_on_pipe = epoch_on_pipe
        self.batch_on_pipe = batch_on_pipe
    
    def dequeue(self): 
        while not self.main_queue.empty(): yield self.main_queue.get()
        
    def __iter__(self):
        epochs = find_pipe_instance(self,self.epoch_on_pipe).epochs if self.epochs is None else self.epochs
        batches = find_pipe_instance(self,self.batch_on_pipe).batches if self.batches is None else self.batches
        mbar = master_bar(list(range(epochs))) 
        pbar = progress_bar(list(range(batches)),parent=mbar,leave=False)
        attached_collectors = {o.name:o.value for o in self.dequeue()}
        mbar.write(attached_collectors, table=True)

        for epoch in mbar:
            for _,batch_step in zip(*(pbar,self.source_datapipe)):
                yield batch_step
                for o in self.dequeue(): attached_collectors[o.name] = o.value
            mbar.write([f'{l:.6f}' if isinstance(l, float) else str(l) 
                        for l in attached_collectors.values()], table=True)            

In [14]:
# export
class RewardCollector(LogCollector):
    def __iter__(self):
        for q in self.main_queues: q.put(Record('reward',None))
        for steps in self.source_datapipe:
            if isinstance(steps,dp.DataChunk):
                for step in steps:
                    for q in self.main_queues: q.put(Record('reward',step.reward.detach().numpy()[0]))
            else:
                for q in self.main_queues: q.put(Record('reward',steps.reward.detach().numpy()[0]))
            yield steps

In [15]:
import pandas as pd
from fastrl.envs.gym import *

envs = ['CartPole-v1']*10

logger_base = ProgressBarLogger(epochs=15,batches=5)

pipe = dp.map.Mapper(envs)
pipe = TypeTransformLoop(pipe,[GymTypeTransform])
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle(count=(18*len(envs))) 
# Turn off the seed so that some envs end before others...
pipe = GymStepper(pipe,synchronized_reset=True)

pipe = RewardCollector(pipe,[logger_base])

pipe = logger_base.connect_source_datapipe(pipe)
# pipe = add_cbs_to_pipes(pipe,L(cb))

steps = list(logger_base)

reward
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [16]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
